In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
class QLearning():
    def __init__(self, qtable_shape):
        self.frames = []
        self.QTable = np.zeros(qtable_shape)
        self.epsi

Episode 39/40 finished after 12 timesteps